In [1]:
import plotly.plotly as py
from plotly.offline import download_plotlyjs, init_notebook_mode, plot,iplot
import plotly.tools
import plotly.graph_objs as go
import numpy as np
import pandas as pd
import skimage
import Images
import ModelUNet_rep
import ModelUNet_v2 #version 2 learns areas and boundaries of nuclei
import matplotlib.pyplot as plt

import copy


/home/lienhard/.local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning:

Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.

Using TensorFlow backend.


In [2]:
train=Images.Images("../input/stage1_train")

#set aside 10% for validation
val1=train.subset(np.arange(train.n()*.9, train.n()))
train=train.subset(np.arange(train.n()*.9,))

model2=ModelUNet_v2.ModelUNet(name='unet_256x256_v2')
model1=ModelUNet_rep.ModelUNet(name='unet_v1_256x256')



found area model file unet_256x256_v2_area.h5
found boundary model file unet_256x256_v2_boundary.h5
found model file


In [3]:
val1.load_images()
val1.load_masks()
val2=copy.deepcopy(val)

val1.add_predictions(model1)
print("expected LB score of v1 (val): {}".format(np.mean(val1.features['iou_score'])))

val2.add_predictions(model2)
val2.features.drop(['ids'], axis=1).head()
print("expected LB score(val): {}".format(np.mean(val2.features['iou_score'])))
#somehow its mutch worse than that...

100%|██████████| 67/67 [00:07<00:00,  8.63it/s]


NameError: name 'val' is not defined

In [ ]:
df=val.features
df['n']=df.index
df['description']=pd.Series("IOU: " + (df.iou_score).astype(str)+"<br>ID: " + (df.n).astype(str))
df_bw=df[(df.n_channels==1)]
df_col=df[(df.n_channels==3)]
df.head()

In [ ]:
trace_bw=go.Scatter(x=df_bw.nuclei_n, y=df_bw.n_pred, mode='markers', marker=dict(size=12, line=dict(width=1), color="black"),
                    name="grayscale", text=df_bw.description,)
trace_col=go.Scatter(x=df_col.nuclei_n, y=df_col.n_pred, mode='markers', marker=dict(size=12, line=dict(width=1), color="red"),
                    name="color", text=df_col.description,)
data=[trace_bw, trace_col]
layout=go.Layout(title="number of nuclei / image", hovermode="closest",
                xaxis=dict(title="# truth", ticklen=5, zeroline=False, gridwidth=2),
                yaxis=dict(title="# prediction", ticklen=5,  gridwidth=2),)
fig=go.Figure(data=data, layout=layout)
plot(fig, show_link=False)

In [ ]:
trace_bw=go.Scatter(x=df_bw.nuclei_n, y=df_bw.iou_score, mode='markers', marker=dict(size=12, line=dict(width=1), color="black"),
                    name="grayscale", text=df_bw.description,)
trace_col=go.Scatter(x=df_col.nuclei_n, y=df_col.iou_score, mode='markers', marker=dict(size=12, line=dict(width=1), color="red"),
                    name="color", text=df_col.description,)
data=[trace_bw, trace_col]
layout=go.Layout(title="number of nuclei / image", hovermode="closest",
                xaxis=dict(title="# truth", ticklen=5, zeroline=False, gridwidth=2),
                yaxis=dict(title="# prediction", ticklen=5,  gridwidth=2),)
fig=go.Figure(data=data, layout=layout)
plot(fig, show_link=False)
#1) the more nuclei the worse
#2) color images are really bad

In [ ]:
fig=tools.make_subplots(rows=1, cols=3, subplot_titles=('masked image','prediction','difference'))

In [ ]:
id=0

data=[dict(type='heatmap', z=val.masks[0],)]
axis=dict(zeroline=False, showgrid=False, ticklen=4)
layout=dict(width=val.features['x'], height=600,
            font=dict(family='Balto', size=12),
            xaxis= dict(axis),
            yaxis= dict(axis),
            title= "mask"
            )
fig1=dict(data=data, layout=layout)
plot(fig1)